<a href="https://colab.research.google.com/github/sj1412/DNN-SummerProject/blob/main/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess the data
iris = load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the labels
y_categorical = tf.keras.utils.to_categorical(y, num_classes=3)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)

# Define a DNN model using Keras
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=100, verbose=0)

# Evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Accuracy: 0.98


In [ ]:
# Convert to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save to file
with open("iris_model.tflite", "wb") as f:
    f.write(tflite_model)

print(" Model converted and saved as 'iris_model.tflite'")


Saved artifact at '/tmp/tmpl8ar7ex5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  136648991618640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136648991619408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136648989345744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136648989344208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136648989346512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136648989347856: TensorSpec(shape=(), dtype=tf.resource, name=None)
 Model converted and saved as 'iris_model.tflite'


In [ ]:
# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="iris_model.tflite")
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Example flower (scaled)
example = scaler.transform([[5.1, 3.5, 1.4, 0.2]]).astype(np.float32)

# Run inference
interpreter.set_tensor(input_details[0]['index'], example)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Display result
predicted_class = np.argmax(output_data)
print(f"Predicted Iris class: {iris.target_names[predicted_class]}")

Predicted Iris class: setosa
